In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/march-machine-learning-mania-2024/Conferences.csv
/kaggle/input/march-machine-learning-mania-2024/sample_submission.csv
/kaggle/input/march-machine-learning-mania-2024/WNCAATourneyDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2024/WRegularSeasonCompactResults.csv
/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySeedRoundSlots.csv
/kaggle/input/march-machine-learning-mania-2024/MRegularSeasonDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2024/MNCAATourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2024/MGameCities.csv
/kaggle/input/march-machine-learning-mania-2024/WGameCities.csv
/kaggle/input/march-machine-learning-mania-2024/MSeasons.csv
/kaggle/input/march-machine-learning-mania-2024/WNCAATourneySlots.csv
/kaggle/input/march-machine-learning-mania-2024/MSecondaryTourneyTeams.csv
/kaggle/input/march-machine-learning-mania-2024/2024_tourney_seeds.csv
/kaggle/input/march-machine-learning-mania-2024/Cities.csv
/

In [3]:
# Other necessary imports for EDA (Exploratory Data Analysis)
import matplotlib.pyplot as plt
import seaborn as sns

## Step 1
### Load in the necessary datasets

In [4]:
# Read in our datasets that we will be using
# Past years NCAA tournament seeds
df_seeds = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySeeds.csv")
# Data on all games through the years including brief statistics of the winning and losing teams performance
df_season_results = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MRegularSeasonDetailedResults.csv")
# Past season's tournament results with brief game statistics
df_tourney_results = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MNCAATourneyCompactResults.csv")
# Importing teams dataframe in case we need to see team names
df_teams = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MTeams.csv")
# Seeding data for 2024 tournament -- need to check if data has been uploaded yet 
df_2024_seeds = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/2024_tourney_seeds.csv')

In [5]:
# Check the columns in the df_seeds dataframe
df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [6]:
# Check to see number of teams each year and make sure all years are accounted for
df_seeds.groupby(by = 'Season')['TeamID'].nunique()

Season
1985    64
1986    64
1987    64
1988    64
1989    64
1990    64
1991    64
1992    64
1993    64
1994    64
1995    64
1996    64
1997    64
1998    64
1999    64
2000    64
2001    65
2002    65
2003    65
2004    65
2005    65
2006    65
2007    65
2008    65
2009    65
2010    65
2011    68
2012    68
2013    68
2014    68
2015    68
2016    68
2017    68
2018    68
2019    68
2021    68
2022    68
2023    68
2024    68
Name: TeamID, dtype: int64

With the addtion of two play-in games we can see there are 68 teams that make the tournament starting in 2011

In [7]:
# Check columns in season results dataframe
df_season_results.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

In [8]:
# Inspect this dataset a little further
pd.set_option('display.max_columns', 35)
df_season_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


The season results dataframe is showing each played and the stats from both the winning and losing teams

In [9]:
# Inspect how many games were played each year
df_season_results.groupby(by='Season').size()

Season
2003    4616
2004    4571
2005    4675
2006    4757
2007    5043
2008    5163
2009    5249
2010    5263
2011    5246
2012    5253
2013    5320
2014    5362
2015    5354
2016    5369
2017    5395
2018    5405
2019    5463
2020    5328
2021    3855
2022    5345
2023    5602
2024    5607
dtype: int64

We see a large drop in game volume in 2021 due to COVID

In [10]:
# Curious to see how the amount of 3PA/G has changed over the years (anticipating a pretty steep increase in last ~10 years)
df_season_results['Total_3PA'] = df_season_results['WFGA3'] + df_season_results['LFGA3'] 
df_season_results.groupby(by='Season')['Total_3PA'].mean()

Season
2003    36.098137
2004    36.395537
2005    36.600642
2006    36.703805
2007    37.592108
2008    38.008910
2009    36.530768
2010    36.139274
2011    36.227221
2012    36.038454
2013    36.118233
2014    36.297091
2015    36.954613
2016    40.795120
2017    41.975904
2018    43.433488
2019    44.578437
2020    43.103041
2021    43.174060
2022    43.407297
2023    42.742949
2024    43.450687
Name: Total_3PA, dtype: float64

Regular season results only go back to 2003

In [11]:
# Inspect columns of the tourney results df
df_tourney_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


Tourney results dataframe gives us a brief statistical description of games played in the NCAA tournament.

This dataframe goes back further than the regular season results

In [12]:
# Checking if this dataframe has results for NCAA tournament in 2020
df_tourney_results.loc[df_tourney_results['Season']==2020]

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT


As we can see there were no tournament games played in the year 2020

In [13]:
# Let's check to see if 2024 seeds have been uploaded 
# *NOTE* This notebook is being created one day after selection sunday for the 2024 NCAA tournament
df_2024_seeds_M = df_2024_seeds.loc[df_2024_seeds['Tournament'] == 'M']
df_2024_seeds_M.head()

,Tournament,Seed,TeamID
0,M,W01,1163
1,M,W02,1235
2,M,W03,1228
3,M,W04,1120
4,M,W05,1361


In [14]:
df_2024_seeds.groupby('Seed').size()

Seed
W01    2
W02    2
W03    2
W04    2
W05    2
      ..
Z12    2
Z13    2
Z14    2
Z15    2
Z16    2
Length: 64, dtype: int64

In [15]:
# Will take a look at the 1 seeds in this years tournament
df_2024_1_seeds = df_2024_seeds_M.loc[df_2024_seeds['Seed'].isin(['W01','X01','Y01','Z01'])]
# Merge the dataframes so we can see names of this years 1 seeds and validate
df_2024_1_seeds.merge(df_teams, on = 'TeamID')

,Tournament,Seed,TeamID,TeamName,FirstD1Season,LastD1Season
0,M,W01,1163,Connecticut,1985,2024
1,M,X01,1314,North Carolina,1985,2024
2,M,Y01,1345,Purdue,1985,2024
3,M,Z01,1222,Houston,1985,2024


Based on the above 1 seeds we can tell the seeding data for this years tournament has been posted

 ## Step 2
### Next, we will work on combining the datasets so that we have one row for each 2024 tournament game with features that are relevant to helping us make predictions on who will win

To Do this we will need to take a few steps.
1. Create a dataframe with one row per game in which we will feature data regarding the winning team
2. Create another dataframe with the inverse data.. Again we will have one row per game, this time we will feature data about the losing team
3. Join these datasets together so we are able to aggregate this data

In [16]:
df_season_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Total_3PA
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20,24
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16,44
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23,44
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23,31
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14,30


In [17]:
# Create dataframe for winning team with desired datapoints
winning_team_df = df_season_results[['Season', 'WTeamID', 'DayNum', 'WScore', 'LScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WTO', 'WStl', 'WOR', 'WDR']]\
.assign(GameResult = 'W')\
.rename(columns = {'WTeamID':'TeamID', 'WScore':'TeamScore', 'LScore':'OppScore', 'WFGM':'FGM','WFGA':'FGA', 'WFGM3':'FGM3','WFGA3':'FGA3', 'WTO':'TO', 'WStl':'Stl', 'WOR':'OR', 'WDR':'DR'})
# Renaming so columns are the same when we concatenate the winning and losing dataframes

In [18]:
# Preview df
winning_team_df.head()

,Season,TeamID,DayNum,TeamScore,OppScore,FGM,FGA,FGM3,FGA3,TO,Stl,OR,DR,GameResult
0,2003,1104,10,68,62,27,58,3,14,23,7,14,24,W
1,2003,1272,10,70,63,26,62,8,20,13,4,15,28,W
2,2003,1266,11,73,61,24,58,8,18,10,5,17,26,W
3,2003,1296,11,56,50,18,38,3,9,12,14,6,19,W
4,2003,1400,11,77,71,30,61,6,14,14,4,17,22,W


In [19]:
# Now let's create a dataframe for the losing team
losing_team_df = df_season_results[['Season', 'LTeamID', 'DayNum', 'LScore', 'WScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LTO', 'LStl', 'LOR', 'LDR']]\
.assign(GameResult = 'L')\
.rename(columns = {'LTeamID':'TeamID', 'LScore':'TeamScore', 'WScore':'OppScore', 'LFGM':'FGM','LFGA':'FGA', 'LFGM3':'FGM3','LFGA3':'FGA3', 'LTO':'TO', 'LStl':'Stl', 'LOR':'OR', 'LDR':'DR'})

In [20]:
# Preview df
losing_team_df.head()

,Season,TeamID,DayNum,TeamScore,OppScore,FGM,FGA,FGM3,FGA3,TO,Stl,OR,DR,GameResult
0,2003,1328,10,62,68,22,53,2,10,18,9,10,22,L
1,2003,1393,10,63,70,24,67,6,24,12,8,20,25,L
2,2003,1437,11,61,73,22,73,3,26,12,2,31,22,L
3,2003,1457,11,50,56,18,49,6,22,19,4,17,20,L
4,2003,1208,11,71,77,24,62,6,16,10,7,21,15,L


In [21]:
# Concatenate the dataframes 
df_team_season_results = pd.concat([winning_team_df, losing_team_df]).reset_index(drop=True)

In [22]:
#Preview new df
df_team_season_results.head()

,Season,TeamID,DayNum,TeamScore,OppScore,FGM,FGA,FGM3,FGA3,TO,Stl,OR,DR,GameResult
0,2003,1104,10,68,62,27,58,3,14,23,7,14,24,W
1,2003,1272,10,70,63,26,62,8,20,13,4,15,28,W
2,2003,1266,11,73,61,24,58,8,18,10,5,17,26,W
3,2003,1296,11,56,50,18,38,3,9,12,14,6,19,W
4,2003,1400,11,77,71,30,61,6,14,14,4,17,22,W


## Step 3: Data Preprocessing & Aggregation

In [23]:
# Create a score differential column
df_team_season_results['ScoreDiff'] = df_team_season_results['TeamScore'] - df_team_season_results['OppScore']
# Create a FG% column
df_team_season_results['FG%'] = df_team_season_results['FGM']/df_team_season_results['FGA']
# Create a FG3% column
df_team_season_results['FG3%'] = df_team_season_results['FGM3']/df_team_season_results['FGA3']
# Create total reboudns column
df_team_season_results['TotReb'] = df_team_season_results['OR'] + df_team_season_results['DR']

In [24]:
# Turn the GameResult column into binary
df_team_season_results['Win'] = (df_team_season_results['GameResult'] == 'W').astype('int')

In [25]:
#Preview winning teams from dataset
df_team_season_results.head()

,Season,TeamID,DayNum,TeamScore,OppScore,FGM,FGA,FGM3,FGA3,TO,Stl,OR,DR,GameResult,ScoreDiff,FG%,FG3%,TotReb,Win
0,2003,1104,10,68,62,27,58,3,14,23,7,14,24,W,6,0.465517,0.214286,38,1
1,2003,1272,10,70,63,26,62,8,20,13,4,15,28,W,7,0.419355,0.400000,43,1
2,2003,1266,11,73,61,24,58,8,18,10,5,17,26,W,12,0.413793,0.444444,43,1
3,2003,1296,11,56,50,18,38,3,9,12,14,6,19,W,6,0.473684,0.333333,25,1
4,2003,1400,11,77,71,30,61,6,14,14,4,17,22,W,6,0.491803,0.428571,39,1


In [26]:
# Preview losing teams from dataset
df_team_season_results.tail()

,Season,TeamID,DayNum,TeamScore,OppScore,FGM,FGA,FGM3,FGA3,TO,Stl,OR,DR,GameResult,ScoreDiff,FG%,FG3%,TotReb,Win
226477,2024,1196,132,67,86,24,66,1,13,11,8,13,26,L,-19,0.363636,0.076923,39,0
226478,2024,1433,132,51,57,17,47,5,24,16,10,9,25,L,-6,0.361702,0.208333,34,0
226479,2024,1458,132,87,93,30,64,7,20,7,7,13,17,L,-6,0.468750,0.350000,30,0
226480,2024,1396,132,69,85,25,61,5,21,14,6,11,26,L,-16,0.409836,0.238095,37,0
226481,2024,1135,132,61,62,23,55,7,25,9,3,8,27,L,-1,0.418182,0.280000,35,0


### Let's now aggregate the data for teams and create some features

In [27]:
df_team_season_agg = (
df_team_season_results.groupby(by = ['Season', 'TeamID']).agg(
    AvgScoreDiff = ('ScoreDiff', 'mean'),
    Wins = ('Win', 'sum'),
    Losses = ('Win', lambda x: (x==0).sum()),
    WinPercentage = ('Win', 'mean'),
    AvgScore = ('TeamScore', 'mean'),
    AvgTO = ('TO', 'mean'),
    AvgFG_Pct = ('FG%' , 'mean'),
    AvgFG3_Pct = ('FG3%', 'mean'),
    AvgReb = ('TotReb', 'mean')
)\
.reset_index())

In [28]:
# Preview new data
df_team_season_agg.head()

,Season,TeamID,AvgScoreDiff,Wins,Losses,WinPercentage,AvgScore,AvgTO,AvgFG_Pct,AvgFG3_Pct,AvgReb
0,2003,1102,0.250000,12,16,0.428571,57.250000,11.428571,0.486149,0.367637,21.000000
1,2003,1103,0.629630,13,14,0.481481,78.777778,12.629630,0.487294,0.331990,29.703704
2,2003,1104,4.285714,17,11,0.607143,69.285714,13.285714,0.419676,0.325442,37.500000
3,2003,1105,-4.884615,7,19,0.269231,71.769231,18.653846,0.396204,0.359630,36.615385
4,2003,1106,-0.142857,13,15,0.464286,63.607143,17.035714,0.425530,0.350196,36.142857


## Step 4: Merge the dataframe we just created with the seeds dataframe

This will allow us to see the seed of each team in the aggregated dataframe(if they made the tournament) 

In [29]:
# First, we need to get just the seed of each team
df_seeds['SeedInt'] = df_seeds['Seed'].str.slice(1)
df_seeds.head()    

,Season,Seed,TeamID,SeedInt
0,1985,W01,1207,01
1,1985,W02,1210,02
2,1985,W03,1228,03
3,1985,W04,1260,04
4,1985,W05,1374,05


In [30]:
# Now we can merge the dataframes
# *NOTE* A lot of the seed values may be null when merged since a large majority of teams do not make the tournament
df_team_season_agg_seed = df_team_season_agg.merge(df_seeds, on = ['Season', 'TeamID'], how = 'left')

In [31]:
# Preview new df
df_team_season_agg_seed.head()

,Season,TeamID,AvgScoreDiff,Wins,Losses,WinPercentage,AvgScore,AvgTO,AvgFG_Pct,AvgFG3_Pct,AvgReb,Seed,SeedInt
0,2003,1102,0.250000,12,16,0.428571,57.250000,11.428571,0.486149,0.367637,21.000000,NaN,NaN
1,2003,1103,0.629630,13,14,0.481481,78.777778,12.629630,0.487294,0.331990,29.703704,NaN,NaN
2,2003,1104,4.285714,17,11,0.607143,69.285714,13.285714,0.419676,0.325442,37.500000,Y10,10
3,2003,1105,-4.884615,7,19,0.269231,71.769231,18.653846,0.396204,0.359630,36.615385,NaN,NaN
4,2003,1106,-0.142857,13,15,0.464286,63.607143,17.035714,0.425530,0.350196,36.142857,NaN,NaN


## We will now go through the same process for tourney results

In [32]:
# Create dataframe for winning team of tourney game
df_tourney_winning_team = df_tourney_results[['Season', 'WTeamID', 'LTeamID', 'WScore', 'LScore']]\
.assign(GameResult = 'W')\
.rename(columns = {"WTeamID":"TeamID", "LTeamID":"OppTeamID", "WScore":"TeamScore", "LScore":"OppScore"})

In [33]:
#Preview df
df_tourney_winning_team

,Season,TeamID,OppTeamID,TeamScore,OppScore,GameResult
0,1985,1116,1234,63,54,W
1,1985,1120,1345,59,58,W
2,1985,1207,1250,68,43,W
3,1985,1229,1425,58,55,W
4,1985,1242,1325,49,38,W
...,...,...,...,...,...,...
2446,2023,1274,1400,88,81,W
2447,2023,1361,1166,57,56,W
2448,2023,1163,1274,72,59,W
2449,2023,1361,1194,72,71,W


In [34]:
# Create df for losing team of tourney game
df_tourney_losing_team = df_tourney_results[['Season', 'LTeamID', 'WTeamID', 'LScore', 'WScore']]\
.assign(GameResult = 'L')\
.rename(columns = {'LTeamID':'TeamID', 'WTeamID':'OppTeamID', 'LScore':'TeamScore', 'WScore':'OppScore'})

In [35]:
# Preview
df_tourney_losing_team.head()

,Season,TeamID,OppTeamID,TeamScore,OppScore,GameResult
0,1985,1234,1116,54,63,L
1,1985,1345,1120,58,59,L
2,1985,1250,1207,43,68,L
3,1985,1425,1229,55,58,L
4,1985,1325,1242,38,49,L


### Append the winning and losing tourney team datasets

In [36]:
df_tourney_results_total = pd.concat([df_tourney_winning_team, df_tourney_losing_team])
df_tourney_results_total.head()

,Season,TeamID,OppTeamID,TeamScore,OppScore,GameResult
0,1985,1116,1234,63,54,W
1,1985,1120,1345,59,58,W
2,1985,1207,1250,68,43,W
3,1985,1229,1425,58,55,W
4,1985,1242,1325,49,38,W


In [37]:
# Create binary win column
df_tourney_results_total['Win'] = (df_tourney_results_total['GameResult']=='W').astype('int')
#Preview
df_tourney_results_total.sample(10)

,Season,TeamID,OppTeamID,TeamScore,OppScore,GameResult,Win
176,1987,1344,1104,103,82,W,1
1532,2009,1140,1401,66,79,L,0
2443,2023,1462,1400,71,83,L,0
2042,2016,1438,1235,84,71,W,1
1984,2016,1435,1455,50,70,L,0
1977,2015,1458,1112,85,78,W,1
304,1989,1278,1181,70,87,L,0
1906,2014,1257,1246,69,74,L,0
2001,2016,1455,1112,65,55,W,1
366,1990,1400,1462,102,89,W,1


## Step 4: Merge aggregated regular season data with tourney data
- This will give us features of the team's regular season performance. These features will help to dictate the outcome of the team in the tournament
- Ultimately March is about match ups.. We want our model to learn patterns in how certain tendencies of one team will fare against tendencies of other teams to determine a winner

In [38]:
df_team_season_agg_seed.head()

,Season,TeamID,AvgScoreDiff,Wins,Losses,WinPercentage,AvgScore,AvgTO,AvgFG_Pct,AvgFG3_Pct,AvgReb,Seed,SeedInt
0,2003,1102,0.250000,12,16,0.428571,57.250000,11.428571,0.486149,0.367637,21.000000,NaN,NaN
1,2003,1103,0.629630,13,14,0.481481,78.777778,12.629630,0.487294,0.331990,29.703704,NaN,NaN
2,2003,1104,4.285714,17,11,0.607143,69.285714,13.285714,0.419676,0.325442,37.500000,Y10,10
3,2003,1105,-4.884615,7,19,0.269231,71.769231,18.653846,0.396204,0.359630,36.615385,NaN,NaN
4,2003,1106,-0.142857,13,15,0.464286,63.607143,17.035714,0.425530,0.350196,36.142857,NaN,NaN


In [39]:
# Merge the data
# We will be merging the data TWICE, this way we get data on the main team AND the opponent
df_historic_tourney_features = df_tourney_results_total.merge(
df_team_season_agg_seed[['Season', 'TeamID', 'AvgScoreDiff', 'WinPercentage', 'AvgScore', 'AvgTO', 'AvgFG_Pct', 'AvgFG3_Pct', 'AvgReb', 'SeedInt']],\
on=['Season', 'TeamID'],\
how='left',)\
.merge(
df_team_season_agg_seed[['Season', 'TeamID', 'AvgScoreDiff', 'WinPercentage', 'AvgScore', 'AvgTO', 'AvgFG_Pct', 'AvgFG3_Pct', 'AvgReb', 'SeedInt']]\
.rename(
        columns = {
            'TeamID': 'OppTeamID',
            'AvgScoreDiff':'OppAvgScoreDiff',
            'WinPercentage':'OppWinPercentage',
            'AvgScore':'OppAvgScore',
            'AvgTO':'OppAvgTO',
            'AvgFG_Pct':'OppAvgFG_Pct',
            'AvgFG3_Pct':'OppAvgFG3_Pct',
            'AvgReb':'OppAvgReb',
            'SeedInt':'OppSeedInt'}),      
on=['Season','OppTeamID']
)

In [40]:
# This cell was run after both merges
df_historic_tourney_features.head()

,Season,TeamID,OppTeamID,TeamScore,OppScore,GameResult,Win,AvgScoreDiff,WinPercentage,AvgScore,AvgTO,AvgFG_Pct,AvgFG3_Pct,AvgReb,SeedInt,OppAvgScoreDiff,OppWinPercentage,OppAvgScore,OppAvgTO,OppAvgFG_Pct,OppAvgFG3_Pct,OppAvgReb,OppSeedInt
0,2003,1421,1411,92,84,W,1,-7.241379,0.448276,71.206897,16.206897,0.432768,0.363494,35.448276,16b,1.966667,0.600000,72.800000,15.233333,0.448892,0.321414,37.966667,16a
1,2003,1112,1436,80,51,W,1,14.964286,0.892857,85.214286,14.785714,0.463563,0.351060,42.821429,01,4.655172,0.655172,67.793103,14.068966,0.446372,0.357920,38.689655,16
2,2003,1113,1272,84,71,W,1,6.793103,0.620690,75.965517,14.000000,0.481680,0.328376,37.000000,10,8.689655,0.793103,74.517241,13.793103,0.439457,0.343441,40.034483,07
3,2003,1141,1166,79,73,W,1,6.103448,0.793103,79.344828,18.241379,0.506349,0.377481,33.862069,11,14.909091,0.878788,79.242424,13.363636,0.498309,0.384914,34.060606,06
4,2003,1143,1301,76,74,W,1,4.724138,0.724138,74.482759,14.172414,0.468741,0.375934,35.620690,08,4.400000,0.600000,72.400000,14.200000,0.458507,0.350564,31.766667,09


In [41]:
# I am seeing some values that need to be cleaned up in the SeedInt and OppSeedInt columns - let's check those
df_historic_tourney_features['SeedInt'].value_counts()

SeedInt
01     323
02     258
03     238
04     202
05     175
07     158
06     146
08     139
09     131
10     123
12     116
11     106
13      97
15      92
14      88
16b     53
16      49
11b     44
16a     44
11a     27
12a      7
12b      6
13b      4
14a      2
14b      1
13a      1
Name: count, dtype: int64

In [42]:
# Remove misc. characters
df_historic_tourney_features['SeedInt'] = df_historic_tourney_features['SeedInt'].str.slice(stop=2)
df_historic_tourney_features['OppSeedInt'] = df_historic_tourney_features['OppSeedInt'].str.slice(stop=2)

In [43]:
#Validate
df_historic_tourney_features.SeedInt.value_counts()

SeedInt
01    323
02    258
03    238
04    202
11    177
05    175
07    158
16    146
06    146
08    139
09    131
12    129
10    123
13    102
15     92
14     91
Name: count, dtype: int64

In [44]:
#Validate
df_historic_tourney_features.OppSeedInt.value_counts()

OppSeedInt
01    323
02    258
03    238
04    202
11    177
05    175
07    158
16    146
06    146
08    139
09    131
12    129
10    123
13    102
15     92
14     91
Name: count, dtype: int64

In [45]:
# Need to turn the seed columns into ints
df_historic_tourney_features['SeedInt'] = df_historic_tourney_features['SeedInt'].astype('int')
df_historic_tourney_features['OppSeedInt'] = df_historic_tourney_features['OppSeedInt'].astype('int')

In [46]:
#Validate datatypes were changed
df_historic_tourney_features.dtypes

Season                int64
TeamID                int64
OppTeamID             int64
TeamScore             int64
OppScore              int64
GameResult           object
Win                   int64
AvgScoreDiff        float64
WinPercentage       float64
AvgScore            float64
AvgTO               float64
AvgFG_Pct           float64
AvgFG3_Pct          float64
AvgReb              float64
SeedInt               int64
OppAvgScoreDiff     float64
OppWinPercentage    float64
OppAvgScore         float64
OppAvgTO            float64
OppAvgFG_Pct        float64
OppAvgFG3_Pct       float64
OppAvgReb           float64
OppSeedInt            int64
dtype: object

In [47]:
# Create some new feautres from this dataset
df_historic_tourney_features['WinPercentageDiff'] = (df_historic_tourney_features['WinPercentage']
                                                     - df_historic_tourney_features['OppWinPercentage'])

df_historic_tourney_features['AvgScoreDiff'] = (df_historic_tourney_features['AvgScore']
                                                - df_historic_tourney_features['OppAvgScore'])

df_historic_tourney_features['AvgFG%Diff'] = (df_historic_tourney_features['AvgFG_Pct'] 
                                              -df_historic_tourney_features['OppAvgFG_Pct'])

df_historic_tourney_features['SeedDiff'] = (df_historic_tourney_features['SeedInt']
                                            - df_historic_tourney_features['OppSeedInt'])

In [48]:
df_historic_tourney_features.head()

,Season,TeamID,OppTeamID,TeamScore,OppScore,GameResult,Win,AvgScoreDiff,WinPercentage,AvgScore,AvgTO,AvgFG_Pct,AvgFG3_Pct,AvgReb,SeedInt,OppAvgScoreDiff,OppWinPercentage,OppAvgScore,OppAvgTO,OppAvgFG_Pct,OppAvgFG3_Pct,OppAvgReb,OppSeedInt,WinPercentageDiff,AvgFG%Diff,SeedDiff
0,2003,1421,1411,92,84,W,1,-1.593103,0.448276,71.206897,16.206897,0.432768,0.363494,35.448276,16,1.966667,0.600000,72.800000,15.233333,0.448892,0.321414,37.966667,16,-0.151724,-0.016123,0
1,2003,1112,1436,80,51,W,1,17.421182,0.892857,85.214286,14.785714,0.463563,0.351060,42.821429,1,4.655172,0.655172,67.793103,14.068966,0.446372,0.357920,38.689655,16,0.237685,0.017191,-15
2,2003,1113,1272,84,71,W,1,1.448276,0.620690,75.965517,14.000000,0.481680,0.328376,37.000000,10,8.689655,0.793103,74.517241,13.793103,0.439457,0.343441,40.034483,7,-0.172414,0.042222,3
3,2003,1141,1166,79,73,W,1,0.102403,0.793103,79.344828,18.241379,0.506349,0.377481,33.862069,11,14.909091,0.878788,79.242424,13.363636,0.498309,0.384914,34.060606,6,-0.085684,0.008041,5
4,2003,1143,1301,76,74,W,1,2.082759,0.724138,74.482759,14.172414,0.468741,0.375934,35.620690,8,4.400000,0.600000,72.400000,14.200000,0.458507,0.350564,31.766667,9,0.124138,0.010234,-1


## This is where we can load in 538 data if we want to improve our model
- FiveThirtyEight creates scores for teams each year
- We will use these as features to improve our model

## Step 5: Create baseline model
- Our benchmark model will be one that takes the better seed to win every game until the final four, in which case all teams involved will be 1 seeds

In [49]:
# Create boolean col of if the main team seed is lower than the opponent seed
df_historic_tourney_features['BaselinePred'] = (
    df_historic_tourney_features['SeedInt']
    < df_historic_tourney_features['OppSeedInt']
)

In [50]:
# Since all teams in final four will have a 1 seed, we will use the greater win percentage to determine the winner
df_historic_tourney_features.loc[
    df_historic_tourney_features['SeedInt'] == df_historic_tourney_features['OppSeedInt'],'BaselinePred']\
    = (df_historic_tourney_features['WinPercentage'] > df_historic_tourney_features['OppWinPercentage'])

In [51]:
# Preview
df_historic_tourney_features

,Season,TeamID,OppTeamID,TeamScore,OppScore,GameResult,Win,AvgScoreDiff,WinPercentage,AvgScore,AvgTO,AvgFG_Pct,AvgFG3_Pct,AvgReb,SeedInt,OppAvgScoreDiff,OppWinPercentage,OppAvgScore,OppAvgTO,OppAvgFG_Pct,OppAvgFG3_Pct,OppAvgReb,OppSeedInt,WinPercentageDiff,AvgFG%Diff,SeedDiff,BaselinePred
0,2003,1421,1411,92,84,W,1,-1.593103,0.448276,71.206897,16.206897,0.432768,0.363494,35.448276,16,1.966667,0.600000,72.800000,15.233333,0.448892,0.321414,37.966667,16,-0.151724,-0.016123,0,False
1,2003,1112,1436,80,51,W,1,17.421182,0.892857,85.214286,14.785714,0.463563,0.351060,42.821429,1,4.655172,0.655172,67.793103,14.068966,0.446372,0.357920,38.689655,16,0.237685,0.017191,-15,True
2,2003,1113,1272,84,71,W,1,1.448276,0.620690,75.965517,14.000000,0.481680,0.328376,37.000000,10,8.689655,0.793103,74.517241,13.793103,0.439457,0.343441,40.034483,7,-0.172414,0.042222,3,False
3,2003,1141,1166,79,73,W,1,0.102403,0.793103,79.344828,18.241379,0.506349,0.377481,33.862069,11,14.909091,0.878788,79.242424,13.363636,0.498309,0.384914,34.060606,6,-0.085684,0.008041,5,False
4,2003,1143,1301,76,74,W,1,2.082759,0.724138,74.482759,14.172414,0.468741,0.375934,35.620690,8,4.400000,0.600000,72.400000,14.200000,0.458507,0.350564,31.766667,9,0.124138,0.010234,-1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2625,2023,1400,1274,81,88,L,0,-1.553309,0.764706,77.852941,11.029412,0.467886,0.333504,31.735294,2,7.281250,0.781250,79.406250,10.937500,0.486964,0.375166,31.875000,5,-0.016544,-0.019078,-3,True
2626,2023,1166,1361,56,57,L,0,5.292614,0.636364,76.636364,10.969697,0.469378,0.359789,34.757576,6,7.531250,0.812500,71.343750,10.906250,0.442519,0.338828,32.656250,5,-0.176136,0.026860,1,False
2627,2023,1274,1163,59,72,L,0,0.951705,0.781250,79.406250,10.937500,0.486964,0.375166,31.875000,5,13.424242,0.757576,78.454545,12.333333,0.459635,0.354549,35.939394,4,0.023674,0.027329,1,False
2628,2023,1194,1361,71,72,L,0,6.812500,0.906250,78.156250,11.093750,0.468408,0.376667,35.187500,9,7.531250,0.812500,71.343750,10.906250,0.442519,0.338828,32.656250,5,0.093750,0.025889,4,False


In [52]:
# Imports for modelling and scoring
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.model_selection import GroupKFold

In [53]:
# Score the benchmark model
benchmark_scores = []

# iterate over the historic tourney features df and return the integer version of the BaselinePred column -- these will be used as our predictions to score against
for season in df_historic_tourney_features['Season'].unique():
    preds =  df_historic_tourney_features.query("Season == @season")['BaselinePred'].astype('int')
    # We will need a true value (y) to comapre our preds to. We will use the win column as the true value
    y = df_historic_tourney_features.query('Season == @season')['Win']
    # Get the accuracy score of our baseline model
    score = accuracy_score(y, preds)
    benchmark_scores.append(score)
    print(f"Holdout season {season} - Accuracy {score:0.4f}")
    
print(f"Baseline accuracy {np.mean(benchmark_scores):0.4f}")

Holdout season 2003 - Accuracy 0.6562
Holdout season 2004 - Accuracy 0.7344
Holdout season 2005 - Accuracy 0.6719
Holdout season 2006 - Accuracy 0.6719
Holdout season 2007 - Accuracy 0.7969
Holdout season 2008 - Accuracy 0.7656
Holdout season 2009 - Accuracy 0.7344
Holdout season 2010 - Accuracy 0.6719
Holdout season 2011 - Accuracy 0.6866
Holdout season 2012 - Accuracy 0.7015
Holdout season 2013 - Accuracy 0.6716
Holdout season 2014 - Accuracy 0.6343
Holdout season 2015 - Accuracy 0.7463
Holdout season 2016 - Accuracy 0.6716
Holdout season 2017 - Accuracy 0.7463
Holdout season 2018 - Accuracy 0.6716
Holdout season 2019 - Accuracy 0.6866
Holdout season 2021 - Accuracy 0.6970
Holdout season 2022 - Accuracy 0.6418
Holdout season 2023 - Accuracy 0.7015
Baseline accuracy 0.6980


## Step 6: Create a model
- Based on the above benchmark model, we want our model to at least score > 69.8%
- We will leave one season out as a validation dataset
- The first model we will try will be an XGBoost Model

In [54]:
# Get the columns from our dataset so we can choose what features we want to use
df_historic_tourney_features.columns

Index(['Season', 'TeamID', 'OppTeamID', 'TeamScore', 'OppScore', 'GameResult',
       'Win', 'AvgScoreDiff', 'WinPercentage', 'AvgScore', 'AvgTO',
       'AvgFG_Pct', 'AvgFG3_Pct', 'AvgReb', 'SeedInt', 'OppAvgScoreDiff',
       'OppWinPercentage', 'OppAvgScore', 'OppAvgTO', 'OppAvgFG_Pct',
       'OppAvgFG3_Pct', 'OppAvgReb', 'OppSeedInt', 'WinPercentageDiff',
       'AvgFG%Diff', 'SeedDiff', 'BaselinePred'],
      dtype='object')

In [55]:
# Need to shuffle our data so that 

In [56]:
# List the features we want our model to use to learn on
feats = ['Season','AvgScoreDiff', 'WinPercentage', 'AvgScore', 'AvgTO',
       'AvgFG_Pct', 'AvgFG3_Pct', 'AvgReb','OppAvgScoreDiff',
       'OppWinPercentage', 'OppAvgScore', 'OppAvgTO', 'OppAvgFG_Pct',
       'OppAvgFG3_Pct', 'OppAvgReb', 'SeedDiff']
target = ['Win','Season']

# Create X and y data
X = df_historic_tourney_features[feats]
y = df_historic_tourney_features[target]


In [57]:
# Create training and test data
X_train = X.loc[X['Season'] < 2023]
y_train = y.loc[y['Season'] < 2023]
X_test = X.loc[X['Season'] >= 2023]
y_test = y.loc[y['Season'] >= 2023]

In [58]:
X.head()

,Season,AvgScoreDiff,WinPercentage,AvgScore,AvgTO,AvgFG_Pct,AvgFG3_Pct,AvgReb,OppAvgScoreDiff,OppWinPercentage,OppAvgScore,OppAvgTO,OppAvgFG_Pct,OppAvgFG3_Pct,OppAvgReb,SeedDiff
0,2003,-1.593103,0.448276,71.206897,16.206897,0.432768,0.363494,35.448276,1.966667,0.600000,72.800000,15.233333,0.448892,0.321414,37.966667,0
1,2003,17.421182,0.892857,85.214286,14.785714,0.463563,0.351060,42.821429,4.655172,0.655172,67.793103,14.068966,0.446372,0.357920,38.689655,-15
2,2003,1.448276,0.620690,75.965517,14.000000,0.481680,0.328376,37.000000,8.689655,0.793103,74.517241,13.793103,0.439457,0.343441,40.034483,3
3,2003,0.102403,0.793103,79.344828,18.241379,0.506349,0.377481,33.862069,14.909091,0.878788,79.242424,13.363636,0.498309,0.384914,34.060606,5
4,2003,2.082759,0.724138,74.482759,14.172414,0.468741,0.375934,35.620690,4.400000,0.600000,72.400000,14.200000,0.458507,0.350564,31.766667,-1


In [59]:
# Drop season from all 
X_train.drop(['Season'], axis = 1, inplace = True)
y_train.drop(['Season'], axis = 1, inplace = True)
X_test.drop(['Season'], axis = 1, inplace = True)
y_test.drop(['Season'], axis = 1, inplace = True)

/tmp/ipykernel_33/2452269603.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(['Season'], axis = 1, inplace = True)
/tmp/ipykernel_33/2452269603.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train.drop(['Season'], axis = 1, inplace = True)
/tmp/ipykernel_33/2452269603.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(['Season'], axis = 1, inplace = True)
/tmp/ipykernel_33/2452269603.py:5: SettingWi

In [60]:
y_train.head()

,Win
0,1
1,1
2,1
3,1
4,1


In [61]:
# Check shapes of training and test data
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2496, 15), (2496, 1), (134, 15), (134, 1))

In [62]:
# Instantiate model
model = xgb.XGBRegressor(eval_metric = 'logloss',
                        n_estimators = 1000,
                        learning_rate = 0.01)

In [63]:
# Fit the model
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='logloss',
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.01, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [64]:
# Make predictions
y_preds = model.predict(X_test)

In [65]:
from sklearn.metrics import log_loss

In [66]:
# Since y_preds are given as a float between 0 and 1 we need to round them
y_preds_round = y_preds.round()

In [67]:
# Preview
y_preds_round

array([ 1.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,
        0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,
        0.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,
        0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,
        1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,
        0.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        1.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,
        1.,  0.,  1.,  0.,  1.,  0., -0.,  0.,  1.,  0.,  0.,  1.,  1.,
        1.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,
        1.,  1.,  0.,  1.], dtype=float32)

In [68]:
# Score the preds
xgb_acc_score = accuracy_score(y_test, y_preds_round)
xgb_acc_score

0.6119402985074627

### Based on the above score (61.19%) we are performing worse than if we were to just take the better seed (as our benchmark model did)

In [69]:
# Let's now try a RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

In [70]:
rfc_model = RandomForestClassifier(n_estimators = 100,
                                   max_depth = 6,
                                   min_samples_split = 3,
                                   n_jobs = -1)

In [71]:
# Fit RandomForestClassifier
rfc_model.fit(X_train, y_train)

/tmp/ipykernel_33/2459394697.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc_model.fit(X_train, y_train)


RandomForestClassifier(max_depth=6, min_samples_split=3, n_jobs=-1)

In [72]:
# Predict
rfc_preds = rfc_model.predict(X_test)

In [73]:
# Score our rfc model
rfc_acc_score = accuracy_score(y_test, rfc_preds)
rfc_acc_score

0.7238805970149254

### Our RandomForestClassifier performed better than our baseline model. We may want to look into how to further improve this model

## Step 6: Predict results of 2024 tournament with our RandomForest model

#### First, we need to pre process the data for 2024 by joining the season aggregates to the 2024 tournament seeds

In [74]:
# Preview 2024 data again
df_2024_seeds_M.head()

,Tournament,Seed,TeamID
0,M,W01,1163
1,M,W02,1235
2,M,W03,1228
3,M,W04,1120
4,M,W05,1361


In [75]:
#  Clean up seed values
df_2024_seeds_M['SeedInt'] = df_2024_seeds_M.Seed.str.slice(1)
df_2024_seeds_M['SeedInt'].value_counts()

/tmp/ipykernel_33/3993685441.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2024_seeds_M['SeedInt'] = df_2024_seeds_M.Seed.str.slice(1)


SeedInt
01    4
02    4
03    4
04    4
05    4
06    4
07    4
08    4
09    4
10    4
11    4
12    4
13    4
14    4
15    4
16    4
Name: count, dtype: int64

### Since we don't know what teams will play each other in later rounds we need top create a dataframe with all potential matchups. 

### We can then use our model to predict these

In [76]:
tourney_pairs = (
    df_2024_seeds_M.merge(df_2024_seeds_M, on = ['Tournament'], suffixes = ("","Opp"))
    .assign(Season = 2024)
    .query("TeamID != TeamIDOpp" )
    .rename(columns = {"Tournament":"League"})
)

In [77]:
tourney_pairs.head()

,League,Seed,TeamID,SeedInt,SeedOpp,TeamIDOpp,SeedIntOpp,Season
1,M,W01,1163,01,W02,1235,02,2024
2,M,W01,1163,01,W03,1228,03,2024
3,M,W01,1163,01,W04,1120,04,2024
4,M,W01,1163,01,W05,1361,05,2024
5,M,W01,1163,01,W06,1140,06,2024


In [97]:
# Similarly to what we did before, we will have to merge twice.

# First, merge to get aggregate stats on main team into df
tourney_pairs_w_season_agg = tourney_pairs.merge(
df_team_season_agg_seed[['Season', 'TeamID', 'AvgScoreDiff', 'WinPercentage', 'AvgScore', 'AvgTO', 'AvgFG_Pct', 'AvgFG3_Pct', 'AvgReb']],\
on=['Season', 'TeamID'],\
how='left',)

In [98]:
# Merage again to get opposing team data into the df
tourney_pairs_w_season_agg = tourney_pairs_w_season_agg.merge(
df_team_season_agg_seed[['Season', 'TeamID', 'AvgScoreDiff', 'WinPercentage', 'AvgScore', 'AvgTO', 'AvgFG_Pct', 'AvgFG3_Pct', 'AvgReb', 'SeedInt']]\
.rename(
        columns = {
            'TeamID': 'TeamIDOpp',
            'AvgScoreDiff':'OppAvgScoreDiff',
            'WinPercentage':'OppWinPercentage',
            'AvgScore':'OppAvgScore',
            'AvgTO':'OppAvgTO',
            'AvgFG_Pct':'OppAvgFG_Pct',
            'AvgFG3_Pct':'OppAvgFG3_Pct',
            'AvgReb':'OppAvgReb'}),      
on=['Season','TeamIDOpp']
)

In [99]:
# Validate 
tourney_pairs_w_season_agg.head()

,League,Seed,TeamID,SeedInt_x,SeedOpp,TeamIDOpp,SeedIntOpp,Season,AvgScoreDiff,WinPercentage,AvgScore,AvgTO,AvgFG_Pct,AvgFG3_Pct,AvgReb,OppAvgScoreDiff,OppWinPercentage,OppAvgScore,OppAvgTO,OppAvgFG_Pct,OppAvgFG3_Pct,OppAvgReb,SeedInt_y
0,M,W01,1163,01,W02,1235,02,2024,17.058824,0.911765,81.470588,9.117647,0.496698,0.364409,35.264706,14.264706,0.794118,75.558824,10.205882,0.463787,0.343456,31.000000,02
1,M,W01,1163,01,W03,1228,03,2024,17.058824,0.911765,81.470588,9.117647,0.496698,0.364409,35.264706,10.617647,0.764706,84.411765,10.294118,0.472623,0.352870,38.058824,03
2,M,W01,1163,01,W04,1120,04,2024,17.058824,0.911765,81.470588,9.117647,0.496698,0.364409,35.264706,15.294118,0.794118,83.323529,10.147059,0.474972,0.351485,34.470588,04
3,M,W01,1163,01,W05,1361,05,2024,17.058824,0.911765,81.470588,9.117647,0.496698,0.364409,35.264706,6.843750,0.687500,74.187500,10.281250,0.440319,0.313812,33.875000,05
4,M,W01,1163,01,W06,1140,06,2024,17.058824,0.911765,81.470588,9.117647,0.496698,0.364409,35.264706,11.939394,0.696970,81.848485,10.363636,0.462849,0.349227,36.212121,06


In [103]:
# Validate
tourney_pairs_w_season_agg.SeedInt_x.value_counts()

SeedInt_x
01    252
02    252
03    252
04    252
05    252
06    252
07    252
08    252
09    252
10    252
11    252
12    252
13    252
14    252
15    252
16    252
Name: count, dtype: int64

In [107]:
# Check datatypes for feature engineering
tourney_pairs_w_season_agg.dtypes

League                object
Seed                  object
TeamID                 int64
SeedInt_x             object
SeedOpp               object
TeamIDOpp              int64
SeedIntOpp            object
Season                 int64
AvgScoreDiff         float64
WinPercentage        float64
AvgScore             float64
AvgTO                float64
AvgFG_Pct            float64
AvgFG3_Pct           float64
AvgReb               float64
OppAvgScoreDiff      float64
OppWinPercentage     float64
OppAvgScore          float64
OppAvgTO             float64
OppAvgFG_Pct         float64
OppAvgFG3_Pct        float64
OppAvgReb            float64
SeedInt_y             object
WinPercentageDiff    float64
AvgFG%Diff           float64
dtype: object

In [110]:
# Change SeedInt_x & SeedIntOpp columns to numeric
tourney_pairs_w_season_agg['SeedInt_x'] = tourney_pairs_w_season_agg['SeedInt_x'].astype('int')
tourney_pairs_w_season_agg['SeedIntOpp'] = tourney_pairs_w_season_agg['SeedIntOpp'].astype('int')

In [111]:
# Feature engineering

tourney_pairs_w_season_agg['WinPercentageDiff'] = (tourney_pairs_w_season_agg['WinPercentage']
                                                     - tourney_pairs_w_season_agg['OppWinPercentage'])

tourney_pairs_w_season_agg['AvgScoreDiff'] = (tourney_pairs_w_season_agg['AvgScore']
                                                - tourney_pairs_w_season_agg['OppAvgScore'])

tourney_pairs_w_season_agg['AvgFG%Diff'] = (tourney_pairs_w_season_agg['AvgFG_Pct'] 
                                              -tourney_pairs_w_season_agg['OppAvgFG_Pct'])

tourney_pairs_w_season_agg['SeedDiff'] = (tourney_pairs_w_season_agg['SeedInt_x']
                                            - tourney_pairs_w_season_agg['SeedIntOpp'])

In [112]:
# Preview
tourney_pairs_w_season_agg.head()

,League,Seed,TeamID,SeedInt_x,SeedOpp,TeamIDOpp,SeedIntOpp,Season,AvgScoreDiff,WinPercentage,AvgScore,AvgTO,AvgFG_Pct,AvgFG3_Pct,AvgReb,OppAvgScoreDiff,OppWinPercentage,OppAvgScore,OppAvgTO,OppAvgFG_Pct,OppAvgFG3_Pct,OppAvgReb,SeedInt_y,WinPercentageDiff,AvgFG%Diff,SeedDiff
0,M,W01,1163,1,W02,1235,2,2024,5.911765,0.911765,81.470588,9.117647,0.496698,0.364409,35.264706,14.264706,0.794118,75.558824,10.205882,0.463787,0.343456,31.000000,02,0.117647,0.032912,-1
1,M,W01,1163,1,W03,1228,3,2024,-2.941176,0.911765,81.470588,9.117647,0.496698,0.364409,35.264706,10.617647,0.764706,84.411765,10.294118,0.472623,0.352870,38.058824,03,0.147059,0.024075,-2
2,M,W01,1163,1,W04,1120,4,2024,-1.852941,0.911765,81.470588,9.117647,0.496698,0.364409,35.264706,15.294118,0.794118,83.323529,10.147059,0.474972,0.351485,34.470588,04,0.117647,0.021727,-3
3,M,W01,1163,1,W05,1361,5,2024,7.283088,0.911765,81.470588,9.117647,0.496698,0.364409,35.264706,6.843750,0.687500,74.187500,10.281250,0.440319,0.313812,33.875000,05,0.224265,0.056380,-4
4,M,W01,1163,1,W06,1140,6,2024,-0.377897,0.911765,81.470588,9.117647,0.496698,0.364409,35.264706,11.939394,0.696970,81.848485,10.363636,0.462849,0.349227,36.212121,06,0.214795,0.033849,-5


In [113]:
# Check to make sure we have all features needed from our 'feats' list we created earlier
tourney_pairs_w_season_agg[feats]

,Season,AvgScoreDiff,WinPercentage,AvgScore,AvgTO,AvgFG_Pct,AvgFG3_Pct,AvgReb,OppAvgScoreDiff,OppWinPercentage,OppAvgScore,OppAvgTO,OppAvgFG_Pct,OppAvgFG3_Pct,OppAvgReb,SeedDiff
0,2024,5.911765,0.911765,81.470588,9.117647,0.496698,0.364409,35.264706,14.264706,0.794118,75.558824,10.205882,0.463787,0.343456,31.000000,-1
1,2024,-2.941176,0.911765,81.470588,9.117647,0.496698,0.364409,35.264706,10.617647,0.764706,84.411765,10.294118,0.472623,0.352870,38.058824,-2
2,2024,-1.852941,0.911765,81.470588,9.117647,0.496698,0.364409,35.264706,15.294118,0.794118,83.323529,10.147059,0.474972,0.351485,34.470588,-3
3,2024,7.283088,0.911765,81.470588,9.117647,0.496698,0.364409,35.264706,6.843750,0.687500,74.187500,10.281250,0.440319,0.313812,33.875000,-4
4,2024,-0.377897,0.911765,81.470588,9.117647,0.496698,0.364409,35.264706,11.939394,0.696970,81.848485,10.363636,0.462849,0.349227,36.212121,-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4027,2024,-2.586918,0.580645,73.774194,11.967742,0.448313,0.342477,34.967742,3.666667,0.611111,76.361111,9.222222,0.452610,0.341307,32.638889,5
4028,2024,-9.195503,0.580645,73.774194,11.967742,0.448313,0.342477,34.967742,13.484848,0.909091,82.969697,10.181818,0.476059,0.362332,34.696970,4
4029,2024,3.117944,0.580645,73.774194,11.967742,0.448313,0.342477,34.967742,6.812500,0.812500,70.656250,8.468750,0.446716,0.335411,31.906250,3
4030,2024,-2.637571,0.580645,73.774194,11.967742,0.448313,0.342477,34.967742,3.500000,0.676471,76.411765,10.852941,0.453031,0.351475,33.323529,2


In [120]:
# Create new features list since we did not use that to train our model
feats = ['AvgScoreDiff', 'WinPercentage', 'AvgScore', 'AvgTO',
       'AvgFG_Pct', 'AvgFG3_Pct', 'AvgReb','OppAvgScoreDiff',
       'OppWinPercentage', 'OppAvgScore', 'OppAvgTO', 'OppAvgFG_Pct',
       'OppAvgFG3_Pct', 'OppAvgReb', 'SeedDiff']

In [122]:
# Make predictions and add a column with the prediction
tourney_pairs_w_season_agg['rfc_pred'] = rfc_model.predict(tourney_pairs_w_season_agg[feats])

In [130]:
# Get data ready for bracket simulation
tourney_pairs_w_season_agg["ID"] = (
    tourney_pairs_w_season_agg["Season"].astype("str")
    + "_"
    + tourney_pairs_w_season_agg["TeamID"].astype("str")
    + "_"
    + tourney_pairs_w_season_agg["TeamIDOpp"].astype("str")
)

In [131]:
# Create a copy that we will use for our bracket builder
preds = tourney_pairs_w_season_agg.copy()

## We now have all necessary predictions for every possible game
- We now want to turn this into the typical bracket format
- To do so we will be using pre-made code below

Code was found here: https://www.kaggle.com/code/lennarthaupts/simulate-n-brackets

In [132]:
from tqdm import tqdm

# Load and filter data
round_slots = pd.read_csv(
    "/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySlots.csv"
)
round_slots = round_slots[round_slots["Season"] == 2024]
round_slots = round_slots[
    round_slots["Slot"].str.contains("R")
]  # Filter out First Four

seeds = pd.read_csv(
    "/kaggle/input/march-machine-learning-mania-2024/2024_tourney_seeds.csv"
)
seeds_m = seeds[seeds["Tournament"] == "M"]
seeds_w = seeds[seeds["Tournament"] == "W"]

preds["ID"] = preds["ID"].str.split("_")

In [135]:
def prepare_data(seeds, preds):
    # Function preparing the data for the simulation
    seed_dict = seeds.set_index("Seed")["TeamID"].to_dict()
    inverted_seed_dict = {value: key for key, value in seed_dict.items()}
    probas_dict = {}

    for teams, proba in zip(preds["ID"], preds["rfc_pred"]):
        team1, team2 = teams[1], teams[2]

        probas_dict.setdefault(team1, {})[team2] = proba
        probas_dict.setdefault(team2, {})[team1] = 1 - proba

    return seed_dict, inverted_seed_dict, probas_dict


def simulate(round_slots, seeds, inverted_seeds, probas, sim=True):
    """
    Simulates each round of the tournament.

    Parameters:
    - round_slots: DataFrame containing information on who is playing in each round.
    - seeds (dict): Dictionary mapping seed values to team IDs.
    - inverted_seeds (dict): Dictionary mapping team IDs to seed values.
    - probas (dict): Dictionary containing matchup probabilities.
    - sim (boolean): Simulates match if True. Chooses team with higher probability as winner otherwise.

    Returns:
    - list: List with winning team IDs for each match.
    - list: List with corresponding slot names for each match.
    """
    winners = []
    slots = []

    for slot, strong, weak in zip(
        round_slots.Slot, round_slots.StrongSeed, round_slots.WeakSeed
    ):
        team_1, team_2 = seeds[strong], seeds[weak]

        # Get the probability of team_1 winning
        proba = probas[str(team_1)][str(team_2)]

        if sim:
            # Randomly determine the winner based on the probability
            winner = np.random.choice([team_1, team_2], p=[proba, 1 - proba])
        else:
            # Determine the winner based on the higher probability
            winner = [team_1, team_2][np.argmax([proba, 1 - proba])]

        # Append the winner and corresponding slot to the lists
        winners.append(winner)
        slots.append(slot)

        seeds[slot] = winner

    # Convert winners to original seeds using the inverted_seeds dictionary
    return [inverted_seeds[w] for w in winners], slots


def run_simulation(brackets=1, seeds=None, preds=None, round_slots=None, sim=True):
    """
    Runs a simulation of bracket tournaments.

    Parameters:
    - brackets (int): Number of brackets to simulate.
    - seeds (pd.DataFrame): DataFrame containing seed information.
    - preds (pd.DataFrame): DataFrame containing prediction information for each match-up.
    - round_slots (pd.DataFrame): DataFrame containing information about the tournament rounds.
    - sim (boolean): Simulates matches if True. Chooses team with higher probability as winner otherwise.

    Returns:
    - pd.DataFrame: DataFrame with simulation results.
    """
    # Get relevant data for the simulation
    seed_dict, inverted_seed_dict, probas_dict = prepare_data(seeds, preds)
    # Lists to store simulation results
    results = []
    bracket = []
    slots = []

    # Iterate through the specified number of brackets
    for b in tqdm(range(1, brackets + 1)):
        # Run single simulation
        r, s = simulate(round_slots, seed_dict, inverted_seed_dict, probas_dict, sim)

        # Update results
        results.extend(r)
        bracket.extend([b] * len(r))
        slots.extend(s)

    # Create final DataFrame
    result_df = pd.DataFrame({"Bracket": bracket, "Slot": slots, "Team": results})

    return result_df


n_brackets = 1
result_m = run_simulation(
    brackets=n_brackets, seeds=seeds_m, preds=preds, round_slots=round_slots, sim=False
)
result_m["Tournament"] = "M"

submission = pd.concat([result_m])
submission = submission.reset_index(drop=True)
submission.index.names = ["RowId"]
submission = submission.reset_index()

100%|██████████| 1/1 [00:00<00:00, 580.93it/s]


In [136]:
submission_w_names = submission.rename(columns={"Team":"Seed"}).merge(
    seeds, on = ['Seed'], how = 'left'
)

teams = pd.read_csv('/kaggle/input/march-machine-learning-mania-2024/MTeams.csv')

submission_w_names = submission_w_names.merge(teams[['TeamID', 'TeamName']], how = 'left')

In [139]:
# Get men's tournament games
mens_submission_w_names = submission_w_names.loc[submission_w_names['Tournament_y'] == 'M']

In [140]:
mens_submission_w_names

,RowId,Bracket,Slot,Seed,Tournament_x,Tournament_y,TeamID,TeamName
0,0,1,R1W1,W01,M,M,1163,Connecticut
2,1,1,R1W2,W02,M,M,1235,Iowa St
4,2,1,R1W3,W03,M,M,1228,Illinois
6,3,1,R1W4,W04,M,M,1120,Auburn
8,4,1,R1W5,W05,M,M,1361,San Diego St
...,...,...,...,...,...,...,...,...
116,58,1,R4Y1,Y01,M,M,1345,Purdue
118,59,1,R4Z1,Z01,M,M,1222,Houston
120,60,1,R5WX,W01,M,M,1163,Connecticut
122,61,1,R5YZ,Y01,M,M,1345,Purdue
